In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
%%time
statements_df = pd.read_csv('statements.csv')
statements_df

Wall time: 37.7 s


,source_item_id,edge_property_id,target_item_id
0,1,31,36906466
1,1,279,3695190
2,1,398,497745
3,1,398,1133705
4,1,398,1139177
...,...,...,...
141206848,77257484,59,9286
141206849,77257491,31,318
141206850,77257491,59,9286
141206851,77257493,31,318


In [3]:
%%time
property_df = pd.read_csv('property.csv')
property_df

Wall time: 36 ms


,property_id,en_label,en_description
0,6,head of government,"head of the executive power of this town, city..."
1,10,video,"relevant video. For images, use the property P..."
2,14,traffic sign,"graphic symbol describing the item, used at th..."
3,15,route map,image of route map at Wikimedia Commons
4,16,highway system,system (or specific country specific road type...
...,...,...,...
6980,7663,Scienza a due voci ID,identifier for an Italian female scientist in ...
6981,7665,FMV World ID,identifier of a video game in the FMV World da...
6982,7666,Visuotinė lietuvių enciklopedija ID,identifier for an item in the online edition o...
6983,7667,Hellenic Civil Aviation Authority airport code,NaN


In [4]:
%%time
property_alias_df = pd.read_csv('property_aliases.csv')
property_alias_df

Wall time: 26 ms


,property_id,en_alias
0,6,mayor
1,6,prime minister
2,6,premier
3,6,first minister
4,6,head of national government
...,...,...
8999,7632,MPPDA Digital Archive organization ID
9000,7650,WeiXin identifier
9001,7668,pdf page
9002,7668,djvu page


In [5]:
edge_groups = statements_df.groupby(['edge_property_id']).count()
edge_groups

,source_item_id,target_item_id
edge_property_id,,
6,22238,22238
16,37003,37003
17,12191946,12191946
19,2245192,2245192
20,857739,857739
...,...,...
7573,3,3
7582,1,1
7600,2,2


In [6]:
property_df['popularity'] = [edge_groups['source_item_id'].get(i, np.nan) for i in property_df.property_id]
property_df = property_df.dropna(subset=['popularity']).sort_values(by=['popularity'], ascending=False).reset_index(drop=True)
property_df

,property_id,en_label,en_description,popularity
0,31,instance of,that class of which this subject is a particul...,26016076.0
1,1433,published in,larger work that a given work was published in...,15581703.0
2,17,country,sovereign state of this item; don't use on humans,12191946.0
3,407,language of work or name,language associated with this creative work (s...,8935493.0
4,131,located in the administrative territorial entity,the item is located on the territory of the fo...,7569815.0
...,...,...,...,...
1213,1033,GHS signal word,"please use with items ""warning"" (Q15350847) an...",1.0
1214,6271,demonym of,item that this sense is a demonym of,1.0
1215,2553,in work,qualifier of award received and nominated for ...,1.0
1216,3774,blocker of,protein (e.g. channel/pore protein) for which ...,1.0


In [7]:
%%time
property_alias_group = property_alias_df.groupby('property_id')['en_alias'].apply(lambda group_series: pd.unique(group_series).tolist())
property_alias_group

Wall time: 584 ms


property_id
6       [mayor, prime minister, premier, first ministe...
10             [media, animation, gif, trailer (Commons)]
14      [motorway sign, shield, highway shield, road m...
15      [street map, road atlas, highway map, schema, ...
16      [network of routes, system of routes, transpor...
                              ...                        
7618                     [Virginia Landmarks Register ID]
7630                                    [DSA building ID]
7632              [MPPDA Digital Archive organization ID]
7650                                  [WeiXin identifier]
7668              [pdf page, djvu page, page of pdf file]
Name: en_alias, Length: 3253, dtype: object

In [8]:
property_df['aliases'] = [property_alias_group.get(i, np.nan) for i in property_df.property_id]
property_df['aliases_count'] = [len(i) if i is not np.nan else np.nan for i in property_df.aliases]
property_df

,property_id,en_label,en_description,popularity,aliases,aliases_count
0,31,instance of,that class of which this subject is a particul...,26016076.0,"[is a, is an, has class, has type, is a partic...",24.0
1,1433,published in,larger work that a given work was published in...,15581703.0,"[on the tracklist of, venue, part of work, pub...",4.0
2,17,country,sovereign state of this item; don't use on humans,12191946.0,"[sovereign state, state, land, host country]",4.0
3,407,language of work or name,language associated with this creative work (s...,8935493.0,"[broadcasting language, audio language, availa...",12.0
4,131,located in the administrative territorial entity,the item is located on the territory of the fo...,7569815.0,"[located in the administrative unit, located i...",46.0
...,...,...,...,...,...,...
1213,1033,GHS signal word,"please use with items ""warning"" (Q15350847) an...",1.0,NaN,NaN
1214,6271,demonym of,item that this sense is a demonym of,1.0,NaN,NaN
1215,2553,in work,qualifier of award received and nominated for ...,1.0,NaN,NaN
1216,3774,blocker of,protein (e.g. channel/pore protein) for which ...,1.0,NaN,NaN


In [9]:
property_df.to_pickle("property_details.p")